<a href="https://colab.research.google.com/github/vgaurav-umich/siads592/blob/master/get_geographic_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Geographic Data 




## Purpose

The purpose of this Notebook is to collect and curate various Geo features, like - Federal Information Processing Standard (FIPS) Codes for various US Counties or Equivelent, their corresponding Core Base Statistical Area (CBSA), type of Statistical Area i.e. Metro or Micro, Population etc. 

We used various Wikipedia articles to download required information.  

In [0]:
# import required libraries
import pandas as pd
import numpy as np

## Load State Data List

In [0]:
# We will be using Wikipedia article to get list of US States or Equivelent
state_url =  "https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code" #@param {type:"raw"}
# It is a good idea to click on this Url and see if this still works

In [0]:
# STEP 1: Load the data table
# First Table in loaded data contains FIPS Codes for States
# This will download list of US States or Equivelent with associated FIPS codes
state_df = pd.read_html(state_url)[0].dropna()

In [0]:
# TEST and DEBUG STEP 1: Check loaded Dataframe
# state_df.info()
# We see that we have 69 records that include main 50 states and union terrirotirs as well as minor islands
# All entries are not null so we do not need to worry about anything 
# state_df.head()

In [0]:
# STEP 2: Clean-up

# Step 2.1 Drop last column which contains status info of a state or territory 
state_df = state_df.iloc[:,0:-1]

#Step 2.2 Rename columns to user friendly names
state_df.columns = ['state', 'state_code', 'fips']

#Step 2.3 Change Virgin Island to friendly Name
state_df['state'] = state_df['state'].str.replace('Virgin Islands of the U.S.', 'Virgin Islands')
# print("\n============================== state_df: State dataset ===========================================")
# state_df.info()

In [0]:
# TEST Step 2: Check Transformed Dataframe
# state_df.info()
# Everything looks good
# Debug to check if it has any null values or a particular entry
# state_df.query('state_code.isnull()', engine = 'python')
# state_df.query('state_code.str.contains("Virgin")', engine = 'python')
# state_df.head()

## Load County Data

In [0]:
# We will use following Wiki article to get list of 3242 US Counties or Equivelents
county_url = "https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county" #@param {type:"raw"}
# It is a good idea to clcik on this Url and see if this still works

In [0]:
# STEP 1: Load the data table
# Second Table contains FIPS for Counties
# This will download list of US Counties or Equivelent with associated FIPS codes
county_df = pd.read_html(county_url)[1]

In [0]:
# TEST and DEBUG Step 1: Check loaded Dataframe
# county_df.info()
# We see that we have 3242 records that include all counties as well as equivelents 
# All entries are not null so we do not need to worry about anything 
# NOTE that it contains old county entry of Valdez-Cordova Census Area 
#  in Alaska which is now split into two census areas. We are going to keep it. 

In [0]:
# STEP 2: Clean-up

# Step 2.1 Rename columns
county_df.columns = ['fips','county', 'state']

# Step 2.2 Let's fix County suffix from each County Name
county_df['county'] = county_df['county'].str.replace(' County', '')
county_df['county'] = county_df['county'].str.replace(' Parish', '')

# Wiki adds some footnotes, so let's remove them
county_df['county'] = county_df['county'].str.replace('(\[[0-9A-Za-z]\])*', '', regex = True)
county_df['county'] = county_df['county'].str.replace(', City and of', '')
county_df['county'] = county_df['county'].str.replace(', City and Borough of', '')
county_df['county'] = county_df['county'].str.replace(', Municipality of', '')
county_df['county'] = county_df['county'].str.replace(', Town and of', '')

# Decided against removing City of suffix for independent cities of Vriginia, 
# becuase removing this suffix was cuasing false duplicate scnearios with County and State Name which was creating problem with later joinig. 
# county_df['county'] = county_df['county'].str.replace(', City of', '')
county_df['county'] = county_df['county'].str.replace(', Consolidated Municipality of', '')
county_df['county'] = county_df['county'].str.replace('–', '-')
county_df['county'] = county_df['county'].str.replace(' – ', '-')
county_df['county'] = county_df['county'].str.replace(' - ', '-')
county_df['state'] = county_df['state'].str.replace('Virgin Islands \(U.S.\)', 'Virgin Islands')
county_df['state'] = county_df['state'].str.replace('Hawaiʻi', 'Hawaii')

In [0]:
# TEST and DEBUG STEP 2: Check Transformed Dataframe\
# print("\n============================== county_df: County dataset ===========================================")
# county_df.info()
# Everything loks good
# Debug to check if any entry exists 
# county_df.query('county.str.contains("New York")', engine = 'python')

## Load County CBSA Data

Here we will load County data for US Counties. This will have MSA, Population, FIPS code, and State codes

In [0]:
# County CBSA mapping page
county_cbsa_url = 'https://en.wikipedia.org/wiki/List_of_United_States_counties_and_county_equivalents' #@param {type:"raw"}

In [0]:
# Step 1: Load Data
#  Forth Table contains US Counties or Equivelent and their associated Metropolitian Statstical Area mapping
county_cbsa_df = pd.read_html(county_cbsa_url)[3]

In [0]:
# Test Step 1: 
# county_cbsa_df.info()
# We see that it has more entries than 3242 we got from County Data pull.
# One reason is Alaska one, not sure what are other reasons
# Note that there are 1874 CBSA represented here
# Debug 
# county_cbsa_df.head()
# Here we see that what need to be transformed

In [0]:
# Step 2: Clean-up
# Let's change column names and datatypes. Note that population is 2019 estimate. 
# Rename of column name might loose this info
county_cbsa_df.columns = ['county','state', 'population', 'cbsa']

# Let's extract cbsa type - Mirco or Metro statstical area designation
county_cbsa_df['cbsa_type'] = county_cbsa_df['cbsa'].str.extract('(Metro|Micro|metro|micro)')
county_cbsa_df['cbsa_type'] = county_cbsa_df['cbsa_type'].str.lower()
# county_cbsa_df['cbsa_type'] = county_cbsa_df['cbsa'].str.extract('.+, [A-Z|-]* (.{5})')
# Test code to test if this worked
# county_cbsa_df[['cbsa', 'cbsa_type']].query("cbsa_type.isna() & cbsa.notnull()", engine = 'python')

# Get rid of Metropolitan Statistical Area suffix
county_cbsa_df['cbsa'] = county_cbsa_df['cbsa'].str.replace('(, [A-Z]{2}.*)', '', regex = True)
# Clean-up County Names to match County names we got earlier
county_cbsa_df['county'] = county_cbsa_df['county'].str.replace(' County', '')
county_cbsa_df['county'] = county_cbsa_df['county'].str.replace(' Parish', '')
# Remove Footnote
county_cbsa_df['county'] = county_cbsa_df['county'].str.replace('(\[[0-9A-Za-z]+\])*', '', regex = True)
county_cbsa_df['county'] = county_cbsa_df['county'].str.replace(', City and of', '')
county_cbsa_df['county'] = county_cbsa_df['county'].str.replace(', Municipality of', '')
county_cbsa_df['county'] = county_cbsa_df['county'].str.replace(', Town and of', '')
# We decided against removing City of suffix for independent cities of Vriginia, 
# becuase removing this suffix was cuasing false duplicate scnearios with County and State Name which was creating problem with later joinig. 
# county_cbsa_df['county'] = county_cbsa_df['county'].str.replace(', City of', '')
county_cbsa_df['county'] = county_cbsa_df['county'].str.replace(', Consolidated Municipality of', '')
county_cbsa_df['county'] = county_cbsa_df['county'].str.replace('–', '-')
county_cbsa_df['county'] = county_cbsa_df['county'].str.replace(' – ', '-')
county_cbsa_df['county'] = county_cbsa_df['county'].str.replace(' - ', '-')
county_cbsa_df['county'] = county_cbsa_df['county'].str.replace(', City and Borough of', '')
county_cbsa_df['state'] = county_cbsa_df['state'].str.replace('Hawaiʻi', 'Hawaii')
county_cbsa_df['state'] = county_cbsa_df['state'].str.replace('Virgin Islands \(U.S.\)', 'Virgin Islands')
county_cbsa_df['county'] = county_cbsa_df['county'].str.replace('(\[[a-z]{2}\])*', '', regex = True)
# Remove Zero population areas. Yes, there are areas with zero population
county_cbsa_df = county_cbsa_df[county_cbsa_df['population'] != 0]
# Finally, recall that in Alska as of Jan 2, 2019 we have two new census areas. This data has that updated, but the old one is dropped. 
# Let's add that back
county_cbsa_df = county_cbsa_df.append({'county':"Valdez-Cordova Census Area", 'state':'Alaska', 'population': 18404, 'cbsa': None,'cbsa_type': None}, ignore_index = True)

In [0]:
# Debug
# Let's check how data looks after changes
# county_cbsa_df.info()
# We see that only 1806 cbsa types, but 1874 cbsa values. what are we mssing in our regex? 
# county_cbsa_df[['cbsa', 'cbsa_type']].query("cbsa_type.isna() & cbsa.notnull()", engine = 'python')
# Let's try to fix that as well
# county_cbsa_df[['cbsa', 'cbsa_type']].query("cbsa_type.isna() & cbsa.notnull()", engine = 'python')
# county_cbsa_df.columns
# county_cbsa_df.columns = ['county','state', 'population', 'cbsa']
# county_cbsa_df.head()

In [0]:
# Step 3: Blend Data with State and County FIPS data so we we will a dataframe with all features
# Step 3.1: Blend with County dataframe which has 3242 entries to get FIPS for each county in MSA area
county_cbsa_full_df = county_cbsa_df.merge(county_df, how = 'inner', on = ['county', 'state'])
county_cbsa_full_df = county_cbsa_full_df.query('population != 0')

county_cbsa_full_df = county_cbsa_full_df.merge(state_df, on = ['state']).rename({'fips_x': 'fips'},axis = 1).drop('fips_y',axis = 1)
print("\n============================== county_cbsa_full_df: Full CBSA and County dataset w/ FIPS and Population ===========================================")
county_cbsa_full_df.info()


============================== county_cbsa_full_df: Full CBSA and County dataset w/ FIPS and Population ===========================================
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3234 entries, 0 to 3233
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   county      3234 non-null   object
 1   state       3234 non-null   object
 2   population  3234 non-null   int64 
 3   cbsa        1874 non-null   object
 4   cbsa_type   1874 non-null   object
 5   fips        3234 non-null   int64 
 6   state_code  3234 non-null   object
dtypes: int64(2), object(5)
memory usage: 202.1+ KB


In [0]:
# Enriched State dataset with population estimate and fips code
state_full_df = county_cbsa_full_df.groupby('state').sum().reset_index().merge(state_df, on = 'state').drop('fips_x', axis = 1).rename({'fips_y': 'fips'}, axis = 1)
print("\n============================== state_full_df: Full State dataset w/ Population and FIPS ===========================================")
state_full_df.info()


============================== state_full_df: Full State dataset w/ Population and FIPS ===========================================
<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 0 to 56
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   state       57 non-null     object
 1   population  57 non-null     int64 
 2   state_code  57 non-null     object
 3   fips        57 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ KB


In [0]:
# state_df.head()

In [0]:
# Enriched State dataset with population estimate and fips code
cbsa_full_df = county_cbsa_full_df.groupby('cbsa').sum().reset_index().drop('fips', axis = 1)
print("\n============================== cbsa_full_df: Full CBSA dataset w/ Population ===========================================")
cbsa_full_df.info()


============================== cbsa_full_df: Full CBSA dataset w/ Population ===========================================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854 entries, 0 to 853
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cbsa        854 non-null    object
 1   population  854 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 13.5+ KB


## Save Data Files

Now that we have captured data for State, County, and CBSA level. Let's Save the Data Files, so we have persistent records. It help us when connection is down or we do not want to reach out to internet everytime we run our analysis.   

In [0]:
state_full_df.to_csv('drive/My Drive/Colab Notebooks/assets/state_full.csv')

In [0]:
county_cbsa_full_df.to_csv('drive/My Drive/Colab Notebooks/assets/county_cbsa_full.csv')

In [0]:
cbsa_full_df.to_csv('drive/My Drive/Colab Notebooks/assets/cbsa_full.csv')